<a href="https://colab.research.google.com/github/otzhora/python_learning/blob/master/Do_deep_conv_nets_needs_to_be.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import os

import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
batch_size = 128
num_classes = 10
epochs = 250
data_augumentations = True
num_predictions = 20

In [0]:
def get_model(x_train):
  model = Sequential()
  model.add(Conv2D(96, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(96, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(171, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(171, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25)) 

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  print(model.summary())
  return model

In [0]:
def get_data():
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  return (x_train, y_train), (x_test, y_test)

In [0]:
def train_model():
  (x_train, y_train), (x_test, y_test) = get_data()
  x_val, y_val = x_train[40000:], y_train[40000:]
  x_train, y_train = x_train[:40000], y_train[:40000]

  model = get_model(x_train)
  datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0)
  datagen.fit(x_train)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
  # Fit the model on the batches generated by datagen.flow().
  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_val, y_val),
                      workers=1,
                      callbacks=[es])
  return model

In [25]:
model = train_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_11 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 96)        83040     
_________________________________________________________________
activation_12 (Activation)   (None, 30, 30, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 96)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 171)      

In [0]:
(x_train, y_train), (x_test, y_test) = get_data()

In [0]:
scores = model.evaluate(x_test, y_test, verbose=0, batch_size=512)

In [31]:
scores

[0.4682715835571289, 0.8586]

In [0]:
model.save("./drive/My Drive/teacher_model.h5")

In [0]:
preds = model.predict(x_train[0:1])

In [46]:
preds[0]

array([8.8463061e-08, 5.6787155e-07, 9.2108050e-05, 3.5279058e-04,
       3.5751124e-05, 2.1839807e-05, 9.9949658e-01, 1.4618507e-08,
       4.7920995e-08, 2.7771165e-07], dtype=float32)